In [3]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [4]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("주식분석")

LangSmith 추적을 시작합니다.
[프로젝트명]
주식분석


In [2]:
# 먼저 필요한 라이브러리를 모두 임포트하자.
import os
from pydantic import BaseModel
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI

In [5]:
# State 정의

# START로부터 None의 입력을 받게 된다.
# 따라서 실제 필요한 인자는 아무것도 없어도 되지만
# Pass나 ... 을 입력해 class를 정의하면 에러가 발생한다.
class InputState(TypedDict):
    start_input: str

# user_input을 받는 Node를 다음과 같이 작성하였다.
# start_input을 입력받아 (실제로는 아무것도 입력되지 않음)
# 아래의 기능을 수행하고, "user_input"이 담긴 Dict를 반환한다.
# 다음 Node의 경우 이러한 변수를 가진 State를 가지고 있어야 한다.
def user_input_node(state: InputState):
    print("================================= Make Persona =================================")
    print("페르소나를 결정합니다. 성별, 나이, 거주지, 취미 등 정보를 알려주세요.")
    # time.sleep(1)
    user_input = input("User: ")
    
    return {"user_input": user_input}

In [6]:
# 이전 Node에서 user_input을 전달하므로 이러한 변수가 반드시 포함되어야 한다.
# 이후 데이터를 만들어 chracter_persona_dict에 저장하려고 한다.
class PersonaState(TypedDict):
    user_input: str
    character_persona_dict: dict

# 노드 생성 전, LLM을 설계해보자.
# 해당 노드에서 작동하는 LLM은 다음과 같은 정보를 반환하도록 만들었다.
# Sturctured_output을 만들기 위해 아래와 같이 작성하였다.
class Persona_Output(TypedDict):
    """
    Sturctured_output을 생성하기위한 클래스
    """
    character_age: Annotated[str, ..., "An age of the Persona"]
    character_sex: Annotated[str, ..., "A sex of the Persona"]
    character_location: Annotated[str, ..., "A place where the persona might live"]
    character_interest: Annotated[str, ..., "Interests that the persona might have"]
    character_hobby: Annotated[str, ..., "Hobbies that the persona might have"]
    
persona_model = ChatOpenAI(model="gpt-4o-mini")
persona_model = persona_model.with_structured_output(Persona_Output)

# 이를 활용하는 노드를 다음과 같이 설계하였다.
# System Prompt를 통해 작업을 지시하였다. 만약 정확한 정보가 제시되지 않으면 임의의 값을 채워넣도록 요청했다.
def persona_setup_node(state: PersonaState):
    messages = [
        ("system", """
         You are the expert in determining your character's persona.
        Extract the character's 'age', 'sex, 'location', 'interest', and 'hobbies' from the values entered by the user.
        If no information is available, it will return a randomised set of appropriate information that must be entered.
        Answers must be in Korean.
        """),
        ("human", state['user_input'])
    ]
    response = persona_model.invoke(messages)
    
    print("================================= Persona Setup =================================")
    print(f"입력된 정보:{state['user_input']}")
    print(f"성별: {response['character_sex']}")
    print(f"나이: {response['character_age']}")
    print(f"거주지: {response['character_location']}")
    print(f"흥미: {response['character_interest']}")
    print(f"취미: {response['character_hobby']}")
    
    return {"character_persona_dict": response}

In [15]:
# 그래프 전체적으로 사용할 State를 정의
class OverallState(TypedDict):
    user_input: str
    messages: Annotated[list, add_messages]
    character_persona_dict: dict

# 그래프를 만드는 builder를 정의. input을 지정해주지 않으면
# OverallState를 START에서 Input으로 요구하게 됨.
graph_builder = StateGraph(OverallState, input=InputState)

# 그래프의 Node를 추가함. 노드의 이름과 노드 함수를 인자로 받음.
graph_builder.add_node("User Input", user_input_node)
graph_builder.add_node("Persona Setup", persona_setup_node)
graph_builder.add_node("Search Sentence", search_setence_node) # 새로 만든 노드를 추가하자.


# 그래프의 Edge를 추가함. 시작과 끝은 항상 START에서 END로 가야함
graph_builder.add_edge(START, "User Input")
graph_builder.add_edge("User Input", "Persona Setup")
graph_builder.add_edge("Persona Setup", "Search Sentence") # 노드에 순서에 맞추어 엣지를 조금 변경하자.
graph_builder.add_edge("Search Sentence", END) # 이제 문장을 생성한 뒤 마무리 되어야 한다.
graph_builder.add_conditional_edges(
    "Query Check", 
    select_next_node, 
    {"is_revise": "Query Revise Tool", END: END}
    )

# 해당 그래프를 컴파일
graph = graph_builder.compile()

# 해당 그래프의 도식을 그려서 저장
with open("graph_output.png", "wb") as f:
    f.write(graph.get_graph().draw_mermaid_png())

config = {"configurable": {"thread_id": "1"}}

# 그래프 호출. 아까 이야기 했듯 start_input에는 아무것도 입력하지 않음.
# 여기서도 마찬가지로 dict형태의 입력이 요구됨.
graph.invoke({"start_input": ""})

ValueError: Found edge starting at unknown node 'Query Check'

In [8]:
# 주어진 정보를 통해 문장을 생성하는 역할을 수행할 것이다.
# 문장을 생성 후, 검증, 재작성 등의 작업을 수행하게 될 것이기 때문에
# Agent의 기능을 위한 messages와, 다른 변수들도 추가로 정의하였다.
class SearchQueryState(TypedDict):
    messages: Annotated[list, add_messages]
    character_persona_dict: dict
    query_list: list
    is_revise: bool
    
# 노드를 정의하기에 앞서 마찬가지로 견고한 출력을 위해 Parser를 작성하였다.

class Search_Output(TypedDict):
    """
    Sturctured_output을 생성하기위한 클래스
    """
    query_list: Annotated[list, ..., "List of queries that customers have entered in your shop"]

search_model = ChatOpenAI(model="gpt-4o")
search_model = search_model.with_structured_output(Search_Output)

# 출력의 성능을 보장하기 위해 LangChain에서 제공하는 FewshotPrompt를 작성하였다.
# 1-Shot 정보가 제공되지 않는 경우 성능이 상당히 떨어지는 모습이 관찰된다.
examples = [
    {"input": 
        """
            User Sex: 여자,
            User Age: 20대,
            User Location: 서울 강남,
            User Interest: 최신 화장법,
            User Hobby: 공원 산책
        """, 
    "output": 
        ['피부진정용 필링패드', '수분에센스', '스틱형 파운데이션', '강아지 간식', '강아지용 배변패드', '강아지 장난감']
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 위에서 정의된 기능들을 추가하여 문장을 생성하는 Node를 설계하였다.
def search_setence_node(state: SearchQueryState):
    prompt = ChatPromptTemplate.from_messages([
        ("system","""
        You're a great marketing manager, and you're working on inferring customer search queries.
        Given the customer information, generate appropriate search quries that customers might enter to find products in your shopping mall.
        Make sure to clearly present the actual product names that a user with that persona would search for in your retail mall.
        """),
        few_shot_prompt,
        ("human", """
         User Sex: {sex},
         User Age: {age},
         User Location: {location},
         User Interest: {interest},
         User Hobby: {hobby}
         """),
    ])
    
    chain = prompt | search_model
    response = chain.invoke(
        {
            "sex": state['character_persona_dict']['character_sex'],
            "age": state['character_persona_dict']['character_age'],
            "location": state['character_persona_dict']['character_location'],
            "interest": state['character_persona_dict']['character_interest'],
            "hobby": state['character_persona_dict']['character_hobby'],
        }
    )
    print("=============================== Search Queries ===============================")
    print(response['query_list'])
    
    return {"query_list": response}

In [10]:
# BaseModel을 이용한 커스텀 툴의 구현.
class QueryReviseAssistance(BaseModel):
    """Escalate the conversation. 
    Use only if the given search query is a strong mismatch with the customer's information.
    Use this tool even if given search query is seriously inappropriate to enter into the search bar of an online retailer like Amazon.
    Never call the tool if the same input is still being given as before.
    To use this function, return 'query_list'.
    """
    query_list: list
    
query_check_model = ChatOpenAI(model="gpt-4o-mini")
query_check_model = query_check_model.bind_tools([QueryReviseAssistance])

In [11]:
def query_check_node(state: SearchQueryState):
    print("=============================== Query Check ===============================")
    prompt = ChatPromptTemplate.from_messages([
        ("system","""
        You are a search manager.
        Based on a given customer persona, if you think that customer would search for the given queries, return the given queries as a list.
        Never call the tool if the same input is still being given as before.
        """),
        ("human", """
            User Sex: {sex},
            User Age: {age},
            User Location: {location},
            User Interest: {interest},
            User Hobby: {hobby}
            Queries: {queries}
            """),
        ])
    chain = prompt | query_check_model
    
    response = chain.invoke(
        {
            "sex": state['character_persona_dict']['character_sex'],
            "age": state['character_persona_dict']['character_age'],
            "location": state['character_persona_dict']['character_location'],
            "interest": state['character_persona_dict']['character_interest'],
            "hobby": state['character_persona_dict']['character_hobby'],
            "queries": state['query_list']['query_list'],
        }
    )
    is_revise = False
        
    if (
        response.tool_calls
        and response.tool_calls[0]["name"] == QueryReviseAssistance.__name__
    ):
        print("Revise Requires")
        is_revise = True
    
    return {"messages": [response], "is_revise": is_revise}

In [12]:
# 마찬가지로 리스트 형태의 반환을 위한 Output Parser
# 이전에 사용한 것과 거의 똑같지만, 구분을 위해 새로 하나를 더 정의함
class QueryCheck_Output(TypedDict):
    """
    Sturctured_output을 생성하기위한 클래스
    """
    query_list: Annotated[list, ..., "List of queries that customers might have entered in search-bar of your online retail shop"]
    
query_revise_model = ChatOpenAI(model="gpt-4o")
query_revise_model = query_revise_model.with_structured_output(QueryCheck_Output)

# 해당 기능을 수행하는 node 설정
def query_revise_node(state: SearchQueryState):
    print("=============================== Query Revise ===============================")
    prompt = ChatPromptTemplate.from_messages([
        ("system",
            """
                You are a validator who fixes errors in a given query.
                From the list of queries given, remove or modify the queries that do not match the user's information appropriately.
                Be sure to delete highly irrelevant data.
                Be sure to remove search terms that you wouldn't use on a shopping site like Amazon.
                Return the modified queries as a list.
            """
        ),
        ("human", 
            """
                User Sex: {sex},
                User Age: {age},
                User Location: {location},
                User Interest: {interest},
                User Hobby: {hobby}
                Queries: {queries}
            """
        )])
    
    chain = prompt | query_revise_model
    response = chain.invoke(
        {
            "sex": state['character_persona_dict']['character_sex'],
            "age": state['character_persona_dict']['character_age'],
            "location": state['character_persona_dict']['character_location'],
            "interest": state['character_persona_dict']['character_interest'],
            "hobby": state['character_persona_dict']['character_hobby'],
            "queries": state['query_list'],
        }
    )
    
    print(response['query_list'])
    
    return {"query_list": response, "is_revise": False}

In [13]:
# 커스텀 라우팅 함수 추가.
def select_next_node(state: SearchQueryState):
    if state["is_revise"]:
        return "is_revise"
    
    return tools_condition(state)